##1.  Use the file bank.csv to explore the dataset. Observe the features:
Are they numbers? Are they strings? Are they binary? Are they continuous?

In [1]:
import pandas as pd
from sklearn import preprocessing
import argparse
import numpy as np
import scipy
import csv
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
from time import time
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier #For #Question 3.
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.neural_network import BernoulliRBM
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.grid_search import ParameterGrid
import pylab as pl
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.grid_search import GridSearchCV
from sklearn.grid_search import ParameterGrid
from scipy import interp
from sklearn.cross_validation import StratifiedKFold

In [4]:
# Load Bank Data subset:
bank_data = pd.read_csv('bank.csv', sep = ";")
# Since data is separated by ";"

In [5]:
bank_data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


The values for features have numbers and strings. 

Some features are binary such as "marital" status and some features are continuous such as "age". 

##2. Learn about label encoders at the following link and use what you learn to transform the features to numerical features.
OneHotEncoder

In [6]:
# Transform the features to numerical features
lab = preprocessing.LabelEncoder()
bank_data.job = lab.fit_transform(bank_data.job)
bank_data.marital = lab.fit_transform(bank_data.marital)
bank_data.education = lab.fit_transform(bank_data.education)
bank_data.default = lab.fit_transform(bank_data.default)
bank_data.housing = lab.fit_transform(bank_data.housing)
bank_data.loan = lab.fit_transform(bank_data.loan)
bank_data.contact = lab.fit_transform(bank_data.contact)
bank_data.month = lab.fit_transform(bank_data.month)
bank_data.poutcome = lab.fit_transform(bank_data.poutcome)
bank_data.y = lab.fit_transform(bank_data.y)

In [7]:
bank_data

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,10,1,0,0,1787,0,0,0,19,10,79,1,-1,0,3,0
1,33,7,1,1,0,4789,1,1,0,11,8,220,1,339,4,0,0
2,35,4,2,2,0,1350,1,0,0,16,0,185,1,330,1,0,0
3,30,4,1,2,0,1476,1,1,2,3,6,199,4,-1,0,3,0
4,59,1,1,1,0,0,1,0,2,5,8,226,1,-1,0,3,0
5,35,4,2,2,0,747,0,0,0,23,3,141,2,176,3,0,0
6,36,6,1,2,0,307,1,0,0,14,8,341,1,330,2,1,0
7,39,9,1,1,0,147,1,0,0,6,8,151,2,-1,0,3,0
8,41,2,1,2,0,221,1,0,2,14,8,57,2,-1,0,3,0
9,43,7,1,0,0,-88,1,1,0,17,0,313,1,147,2,0,0


##3. Build a decision tree model *to predict whether a prospect will buy the product.*

In [11]:
bank_data.columns
# OR "columns = bank_data.columns[1:]"

Index([u'age', u'job', u'marital', u'education', u'default', u'balance', u'housing', u'loan', u'contact', u'day', u'month', u'duration', u'campaign', u'pdays', u'previous', u'poutcome', u'y'], dtype='object')

In [12]:
# use DecisionTreeClassifier
# "from sklearn.tree import DecisionTreeClassifier"
feature_cols = [u'age', u'job', u'marital', u'education', u'default', u'balance', u'housing', u'loan', u'contact', u'day', u'month', u'duration', u'campaign', u'pdays', u'previous', u'poutcome']
X = bank_data[feature_cols]
y = bank_data.y

In [13]:
# split into train/test
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [14]:
# print out each of the arrays
print X_train
print y_train
print X_test
print y_test

[[ 52   4   2 ...,  -1   0   3]
 [ 44   1   1 ..., 357   1   0]
 [ 55   1   1 ...,  -1   0   3]
 ..., 
 [ 53   1   1 ..., 181   1   2]
 [ 32   1   1 ...,  -1   0   3]
 [ 45   9   1 ...,  -1   0   3]]
[0 0 0 ..., 1 0 0]
[[35  1  1 ..., -1  0  3]
 [31  4  2 ..., -1  0  3]
 [30 10  1 ..., -1  0  3]
 ..., 
 [44  4  1 ..., -1  0  3]
 [41  2  1 ..., -1  0  3]
 [41  1  2 ..., -1  0  3]]
[0 0 0 ..., 0 1 0]


In [15]:
# function "DecisionTreeClassifier" for instatiate estimator: http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html
dtc = DecisionTreeClassifier(random_state=0)
dtc.fit(X_train, y_train)

DecisionTreeClassifier(compute_importances=None, criterion='gini',
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_density=None, min_samples_leaf=1, min_samples_split=2,
            random_state=0, splitter='best')

In [32]:
# make predictions
predictions = dtc.predict(X_test)

# print predictions and actual values
print predictions
print y_test

[0 0 0 ..., 0 0 0]
[0 0 0 ..., 0 1 0]


In [28]:
# Check accuracy
float(sum(predictions == y_test)) / len(y_test)

0.870026525198939

##4. Evaluate the accuracy of your decision tree model using cross validation.

In [33]:
from sklearn.cross_validation import cross_val_score
scores = cross_val_score(dtc, X_train, y_train, cv=5)

# cv = k could be standardly 5 or 10, but we can find the k also

In [102]:
print scores
print 'Decision Tree accuracy: {:.2f} %'.format(scores.mean()*100)

[ 0.91390193  0.9150348   0.91568215  0.91500729  0.91808321]
Decision Tree accuracy: 91.55 %


##5. Repeat the analysis and cross validation with the file bank-additional-full.csv. How does the performance of the model change (with the additional training examples and additional features)?

In [39]:
# Load Bank Additional Data:
bank_additional_data = pd.read_csv('bank-additional-full.csv', sep = ";")

In [40]:
# Check the new data head
bank_additional_data.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no


In [41]:
# Use label encoders and transform the features to numerical features column by column

lab = preprocessing.LabelEncoder()
bank_additional_data.job = lab.fit_transform(bank_additional_data.job)
bank_additional_data.marital = lab.fit_transform(bank_additional_data.marital)
bank_additional_data.education = lab.fit_transform(bank_additional_data.education)
bank_additional_data.default = lab.fit_transform(bank_additional_data.default)
bank_additional_data.housing = lab.fit_transform(bank_additional_data.housing)
bank_additional_data.loan = lab.fit_transform(bank_additional_data.loan)
bank_additional_data.contact = lab.fit_transform(bank_additional_data.contact)
bank_additional_data.day_of_week = lab.fit_transform(bank_additional_data.day_of_week)
bank_additional_data.month = lab.fit_transform(bank_additional_data.month)
bank_additional_data.poutcome = lab.fit_transform(bank_additional_data.poutcome)
bank_additional_data.y = lab.fit_transform(bank_additional_data.y)

In [87]:
# Use label encoders and transform ALL the features to numerical features WITHOUT writing the features out as above... 
for col in bank_additional_data:
    bank_additional_data[col] = lab.fit_transform(bank_additional_data[col])

In [88]:
# Look at bank additional data 
bank_additional_data

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,39,3,1,0,0,0,0,1,6,1,...,0,26,0,1,8,18,16,287,8,0
1,40,7,1,3,1,0,0,1,6,1,...,0,26,0,1,8,18,16,287,8,0
2,20,7,1,3,0,2,0,1,6,1,...,0,26,0,1,8,18,16,287,8,0
3,23,0,1,1,0,0,0,1,6,1,...,0,26,0,1,8,18,16,287,8,0
4,39,7,1,3,0,0,2,1,6,1,...,0,26,0,1,8,18,16,287,8,0
5,28,7,1,2,1,0,0,1,6,1,...,0,26,0,1,8,18,16,287,8,0
6,42,0,1,5,0,0,0,1,6,1,...,0,26,0,1,8,18,16,287,8,0
7,24,1,1,7,1,0,0,1,6,1,...,0,26,0,1,8,18,16,287,8,0
8,7,9,2,5,0,2,0,1,6,1,...,0,26,0,1,8,18,16,287,8,0
9,8,7,2,3,0,2,0,1,6,1,...,0,26,0,1,8,18,16,287,8,0


In [43]:
# Look at the column labels
bank_additional_data.columns

Index([u'age', u'job', u'marital', u'education', u'default', u'housing', u'loan', u'contact', u'month', u'day_of_week', u'duration', u'campaign', u'pdays', u'previous', u'poutcome', u'emp.var.rate', u'cons.price.idx', u'cons.conf.idx', u'euribor3m', u'nr.employed', u'y'], dtype='object')

In [94]:
# To build this Decision Tree -> use DecisionTreeClassifier
# "from sklearn.tree import DecisionTreeClassifier"

feature_cols = [u'age', u'job', u'marital', u'education', u'default', u'housing', u'loan', u'contact', u'month', u'day_of_week', u'duration', u'campaign', u'pdays', u'previous', u'poutcome', u'emp.var.rate', u'cons.price.idx', u'cons.conf.idx', u'euribor3m', u'nr.employed']
X_2 = bank_additional_data[feature_cols]
y_2 = bank_additional_data.y

In [95]:
# split into train/test
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_2, y_2, random_state=5)

In [96]:
# print out each of the arrays for the Bank Additional Data
print X_train
print y_train
print X_test
print y_test

[[ 37   1   0 ...,  16 289   8]
 [ 25   9   1 ...,   8 304  10]
 [ 31   7   2 ...,   3 221   6]
 ..., 
 [ 16   9   1 ...,  17 310  10]
 [ 19   0   2 ...,   6 312  10]
 [ 19   1   1 ...,   4 197   6]]
[0 0 1 ..., 0 0 0]
[[ 30   1   2 ...,   8 306  10]
 [ 40   1   1 ...,  16 287   8]
 [ 18   4   1 ...,   3 216   6]
 ..., 
 [ 29   0   1 ...,   3 216   6]
 [ 37   1   1 ...,  17 307  10]
 [ 15   9   1 ...,  17 313  10]]
[0 0 0 ..., 1 0 0]


In [97]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(max_depth=None, n_estimators=100, random_state=5)

# rfc.fit(X_train, y_train)
# NO Need to .fit if doing cross validation

In [98]:
dtc = DecisionTreeClassifier(random_state=5)
scores2 = cross_val_score(dtc, X_train, y_train, cv=5)
scores2

array([ 0.88736041,  0.8889788 ,  0.88525651,  0.88861907,  0.90173223])

In [104]:
print 'Decision Tree accuracy: {:.2f} %'.format(scores2.mean()*100)

Decision Tree accuracy: 89.04 %


In [99]:
scores = cross_val_score(rfc, X_train, y_train, cv=5)
scores

array([ 0.91390193,  0.9150348 ,  0.91568215,  0.91500729,  0.91808321])

In [103]:
print 'Random Forest accuracy: {:.2f} %'.format(scores.mean()*100)

Random Forest accuracy: 91.55 %
